# **Import libraries**

In [3]:
import pandas as pd
import numpy as np
import scipy
import scipy.sparse as sps
import matplotlib.pyplot as plt
!pip install louvain
import louvain
import igraph as ig
import time
import pickle
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.5 MB/s eta 0:00:00
  Using cached texttable-1.7.0-py2.py3-none-any.whl (10 kB)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load Reddit data**

In [5]:
# We want to visualize the graph of words instead of the one of documents, so
#firstly we load the words' occurrences already cleaned and prepped
in_dir = "drive/MyDrive/Colab Notebooks/IP-NS/drive_comments/"
in_file = "reddit_titles_posts_final_nohindi_parent"

Mwd, words, documents = pickle.load(open(in_dir+in_file+"_occurrences.p","rb"))

In [7]:
# List of words that appear without the subsequent label "ADV" or "VERB" etc
WORDS=[w.split()[0] for w in words.copy()]

In [8]:
n=Mwd.shape[0] #number of words like "word LABEL" considered by the occurrency matrix Mwd
m=Mwd.shape[1] #number of documents considered by Mwd
WORDSUNIQUE=[] #introducing the list of unique words in WORDS
Owd=np.zeros((1,m)) #introducing a new occurrency matrix that counts just words in WORDSUNIQUE
cont=0

In [9]:
# Constructing both Owd and WORDSUNIQUE as defined above:
imax=-1

for i in range(imax+1,n):
  if i==cont:
    word=WORDS[i]
    WORDSUNIQUE.append(word)
    for j in range(i,n):
      if word!=WORDS[j]:
        cont=j
        break
      Owd[-1,:]+=Mwd[j,:]
    Owd=np.vstack((Owd,np.zeros((1,m))))
  #if i==8000:  # this snippet of code was added in case Colab was unable to run it all in one go,
                #allowing it to break the process in multiple loops by manually updating imax and
                #rerunning the code each time
  #  break

In [10]:
# Verifying that the last line of Owd is made of zeros:
print((Owd[-1]==np.zeros(m)).sum()==m)

# Deleting the last line of Owd:
Owd=Owd[:-1]

True


# **Build probability matrices from words occurrences**

In [12]:
%run "drive/MyDrive/Colab Notebooks/IP-NS/communities_mod.ipynb"

bertopic 1.11
softlouvain v1.10
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Cleaning occurrency matrix Owd by removing nodes with too high or too low of a degree:
Owd = sps.csr_matrix(Owd)
WORDSUNIQUE = np.array(WORDSUNIQUE)
Owd, WORDSUNIQUE, documents = clean_Mwd_matrix(Owd, WORDSUNIQUE, documents)

removing: change climate  


In [14]:
# Building probability matrices starting from Owd:
POwd, POww, POdd, POaa = probability_matrices(Owd, tform=False)

In [15]:
# Now we want to build on Gephi a graph representing the relationships between words,
#so we set the nodes to be the words and the edges between them to have weights equal
#to the corresponding entries of the POww matrix. Thus, the degree of a node is the
#sum of the entries of the corresponding row of POww.

# Node IDs and labels:
node_ids = list(range(POww.shape[0]))
node_labels = list(WORDSUNIQUE)
node_degrees = [sum((POww[i,:]).toarray())[0] for i in range(POww.shape[0])]

# Combining node IDs, labels and degrees data into a list of tuples:
rows = zip(node_ids, node_labels, node_degrees)

In [16]:
# Specifying the CSV file path:
csv_file_path = in_dir+in_file+'_outputwords1.csv'

R = list(rows)
W=R

In [17]:
# Writing CSV file with all the nodes' data:
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Header:
    csv_writer.writerow(['Id', 'Label','Degree'])

    # Writing data:
    csv_writer.writerows((W))

print("CSV file written successfully")

CSV file written successfully


In [18]:
# Now for the edges, we introduce source and target to be lists storing the nodes' ids
#corresponding to each certain edge, while also saving this edge's weight into the weights list:
source=[]
target=[]
weight=[]
for i in range(POww.shape[0]):
  for j in range(i,POww.shape[0]):
    if POww[i,j]!=0:
      source.append(i)
      target.append(j)
      weight.append(POww[i,j])

In [19]:
# Combining the data we've extracted into a list of tuples:
rows = zip(source, target, weight)
R=list(rows)
W=np.array(W)
nodes_selected=W[:,0] #this is just in case we had previously filtered some more the nodes to be considered
Y=[]
for i in range(len(R)):
  if (str(R[i][0]) in nodes_selected) and (str(R[i][1]) in nodes_selected):
    Y.append(R[i])

In [20]:
# Writing CSV file with all the edges' data:
csv_file_path = in_dir+in_file+'_outputwordsedges.csv'

with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Header:
    csv_writer.writerow(['Source', 'Target', 'Weight'])

    # Writing data:
    csv_writer.writerows(Y)

In [ ]:
# Now we have all the files to correctly visualize the graph on Gephi

# **Assign documents to topics using Louvain and soft Louvain**
i.e., run Louvain community detection on POdd

In [22]:
# start a time counter
tic = time.time()

# build a graph based on Pdd as adjacency matrix
A = sps.csr_matrix(POdd)
G = ig.Graph.Adjacency((A > 0).toarray().tolist())
G.es['weight'] = np.array(A[A.nonzero()])[0]

# run Louvain on the graph to get a partition
part = louvain.find_partition(G, louvain.ModularityVertexPartition,
                                 weights='weight')

# function to map the partition into a community assignment matrix C
# where rows represent documents, and columns represent topics

def partition_to_C(part):
  C = sps.csr_matrix((POdd.shape[0],len(part)))
  for i in range(len(part)):
    C[np.array(part[i]),i] = 1
  return C

# map the partition into a community assignment matrix C
C_l = partition_to_C(part)

# capture execution time
et_louv = time.time()-tic


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [23]:
tic = time.time()
# refine with soft Louvain
C_sl, _, _ = my_soft_louvain(POdd, C_l)

# capture execution time
et_slouv = time.time()-tic

In [24]:
# Checking here that my_soft_louvain actually finds overlapping communities when applied to the (hard)
#community assignment matrix found by the louvain algorithm:

comparisons=[[i for i,j in zip(*C_sl.nonzero()) if j==n] for n in range(C_sl.shape[1])]
setcomparisons=[set(o) for o in comparisons] #this list stores the sets of indices of elements in C_sl
#that are non-zero. Each set correspond to each community found by my_soft_louvain.

comp=[setcomparisons[i].intersection(setcomparisons[j]) for i in range(C_sl.shape[1]) for j in range(i+1,C_sl.shape[1])]
#comp is a list storing the intersections between the sets in setcomparisons, taken two by two.
#Thus if a document was assigned to at least 2 different communities, it will definitely appear in comp.

# Checking which kind of non-empty interesections, if any, are there:
print([o for o in comp if o!=set()])

[]


In [25]:
# Since apparently even C_sl was a hard community assignment matrix, here we try applying
#my_soft_louvain to the trivial community assignment matrix (each document represent 1 community):
C_soft, _, prov = my_soft_louvain(POdd)

In [26]:
# Looking at the intersections of the sets of non zero indices regarding C_soft:
comparisons=[[i for i,j in zip(*C_soft.nonzero()) if j==n] for n in range(C_soft.shape[1])]
setcomparisons=[set(o) for o in comparisons]
comp=[setcomparisons[i].intersection(setcomparisons[j]) for i in range(C_soft.shape[1]) for j in range(i+1,C_soft.shape[1])]

# Checking which kind of non-empty intersections, if any, are there:
print([o for o in comp if o!=set()])

[]


In [27]:
# Seeing that for the dataset we've acquired, my_soft_louvain doesn't find overlapping communities,
#but rather just makes slight adjustments to the C_l found by louvain, the need to actually account for
#overlapping communities arises. Therefore we've decided to implement the algorithm of BigClam that should
#do exactly that. In the end we'll compare the original louvain to BigClam, without considering my_soft_louvain.